# GWExPy 新機能総合チュートリアル

`gwexpy` は `gwpy` を拡張し、多チャンネルデータの効率的な処理 (Matrixクラス)、高度な信号処理 (PCA/ICA)、欠損値処理 (Imputation)、および外部ライブラリとの相互運用性 (Interop) を追加したライブラリです。
このノートブックでは、`gwexpy` で追加された主要な機能を一通り紹介します。

In [ ]:
import numpy as np
from astropy import units as u
from gwpy.time import LIGOTimeGPS

# GWExPyの主要クラスをインポート
from gwexpy.timeseries import TimeSeries, TimeSeriesMatrix

# 乱数シードの固定
np.random.seed(42)

## 1. TimeSeriesMatrix: 多チャンネルデータの効率的処理

`TimeSeriesMatrix` は、複数の時系列データ（チャンネル）を1つの行列として扱うためのクラスです。
`gwpy.TimeSeries` のリストや辞書を使うよりも、メモリ効率が良く、一括処理に適しています。

In [ ]:
# データの生成: 3つのチャンネル、1000サンプル
n_channels = 3
n_samples = 1000
rate = 100 * u.Hz
dt = 1 / rate

# 形状は (チャンネル数, 1, 時間) = (3, 1, 1000) とします
data = np.random.randn(n_channels, 1, n_samples)

# TimeSeriesMatrix の作成
tsm = TimeSeriesMatrix(
    data,
    dt=dt,
    t0=0,
    unit="V",
    channel_names=["CH1", "CH2", "CH3"]
)

print("TimeSeriesMatrix Summary:")
print(tsm)
print(f"Shape: {tsm.shape}")

### スライスとアクセス

行列の一部をスライスすると `TimeSeriesMatrix` が返り、1つの要素を指定すると `TimeSeries` オブジェクトが返されます。

In [ ]:
# 最初の2チャンネルを取得
sub_matrix = tsm[:2, :]
print(f"Sub-matrix shape: {sub_matrix.shape}")

# 特定のチャンネル (CH1) を TimeSeries として取得
ts_ch1 = tsm[0, 0]
print("\nExtracted TimeSeries (CH1):")
print(ts_ch1)

## 2. 信号処理: 標準化、白色化、成分分解 (PCA/ICA)

`TimeSeriesMatrix` は、機械学習の前処理によく使われる機能を内蔵しています。

In [ ]:
# データの標準化 (Z-score normalization)
tsm_norm = tsm.standardize(axis="time")
print(f"Standardized mean (approx 0): {tsm_norm.value.mean():.2f}")
print(f"Standardized std (approx 1):  {tsm_norm.value.std():.2f}")

# チャンネル間の白色化 (Whitening)
tsm_w, w_model = tsm.whiten_channels()
print("\nWhitened Matrix shape:", tsm_w.shape)

### 主成分分析 (PCA)
多チャンネルデータの次元削減や特徴抽出に使用できます。

In [ ]:
# PCA の実行
scores, pca_model = tsm.pca(return_model=True, n_components=2)

print("PCA Scores shape:", scores.shape)
print("explained_variance_ratio:", pca_model.explained_variance_ratio_)

## 3. 欠損値処理 (Imputation)

`gwexpy` は `impute()` メソッドにより、欠損値 (NaN) の補間をサポートしています。

In [ ]:
# 欠損値を含むデータを作成
data_nan = data.copy()
data_nan[0, 0, 50:60] = np.nan  # CH1の一部を NaN に

tsm_nan = TimeSeriesMatrix(data_nan, dt=dt, t0=0)

# 線形補間
tsm_imputed = tsm_nan.impute(method="interpolate")

# 確認
original_segment = data[0, 0, 50:60]
imputed_segment = tsm_imputed.value[0, 0, 50:60]

print("Original:", original_segment)
print("Imputed: ", imputed_segment)

## 4. 周波数領域: FrequencySeriesMatrix と Group Delay

`TimeSeriesMatrix` に `fft()` を適用すると `FrequencySeriesMatrix` が得られます。また、`FrequencySeries` には群遅延 (`group_delay`) を計算する機能が追加されています。

In [ ]:
# FFT (Matrix全体)
fsm = tsm.fft()
print("FrequencySeriesMatrix:")
print(fsm)

# 特定のチャンネルの FrequencySeries を取得
fs = fsm[0, 0]

# 群遅延 (Group Delay) の計算
# group_delay = -d(phi)/d(omega)
gd = fs.group_delay()
print("\nGroup Delay:")
print(gd)

## 5. 相互運用性 (Interop) の応用例

最後に、これらの処理を行ったデータを PyTorch や Pandas に渡すワークフローの例を示します。

In [ ]:
try:
    # 前処理済み (PCAなど) のデータを PyTorch Tensor に変換して学習へ
    import torch
    tensor = tsm.to_torch(dtype=torch.float32)
    print("\nReady for Deep Learning (Torch Tensor):", tensor.shape)
    
    # 単一チャンネルを Pandas で解析
    ts_single = tsm[0, 0]
    df = ts_single.to_pandas()
    print("\nReady for Analysis (Pandas):\n", df.head())
    
except ImportError:
    print("Optional dependencies not installed.")

## まとめ

`gwexpy` の新機能を使うことで、多チャンネルデータの管理、前処理、そして他の最新ライブラリとの連携が非常にスムーズになります。